#  MLP Classifier for  SER (Speech Emotion Recognition)

Multilayer perceptrons (MLP) are often applied to supervised learning problems. They train on a set of input-output pairs and learn to model the correlation (or dependencies) between those inputs and outputs. The networkthus has a simple interpretation as a form of input-output model, with the weights and thresholds (biases) the free parameters of the model.

## Dataset

The Ryerson Audio-Visual Database of Emotional Speech and Song (RAVDESS),and it is free to download. This dataset has 7356 files rated by 247 individuals 10 times on emotional validity, intensity, and genuineness. here the Speech files of all actors (01-24) will be used and the files are available under path ../datasets/RAVDESS. It contains 1440 files: 60 trials per actor x 24 actors = 1440. 
Filename identifiers:
<ol>
<li>Modality (01 = full-AV, 02 = video-only, 03 = audio-only).</li>
<li>Vocal channel (01 = speech, 02 = song).</li>
<li>Emotion (01 = neutral, 02 = calm, 03 = happy, 04 = sad, 05 = angry, 06 = fearful, 07 = disgust, 08 = surprised).</li>
<li>Emotional intensity (01 = normal, 02 = strong). NOTE: There is no strong intensity for the 'neutral' emotion.</li>
<li>Statement (01 = "Kids are talking by the door", 02 = "Dogs are sitting by the door").</li>
<li>Repetition (01 = 1st repetition, 02 = 2nd repetition)..</li>
<li>Actor (01 to 24. Odd numbered actors are male, even numbered actors are female).</li>
</ol>

## Downloading Libraries

In [1]:
#!pip install librosa soundfile numpy sklearn pyaudio

In [2]:
#!pip install soundfile

## Libraries Import

In [3]:
# Import libraries
import librosa
import librosa.display
import json
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from matplotlib.pyplot import specgram
import os, glob, pickle
import IPython.display as ipd 
from tqdm import tqdm

import soundfile
import seaborn as sns
import pandas as pd

# Keras
import keras
from keras import regularizers
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model, model_from_json
from keras.layers import Dense, Embedding, LSTM
from keras.layers import Input, Flatten, Dropout, Activation, BatchNormalization
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D
from keras.utils import np_utils, to_categorical
from keras.callbacks import ModelCheckpoint


# sklearn

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

## Feature Extraction

Defining a function extract_feature to extract the mfcc, chroma, and mel features from a sound file. This function takes 4 parameters- the file name and three Boolean parameters for the three features:
file.
<ol>
<li>file: for the given file name with the path</li>
<li>mfcc: Mel Frequency Cepstral Coefficient, represents the short-term power spectrum of a sound</li>
<li>chroma: Pertains to the 12 different pitch classes</li>
<li>mel: Mel Spectrogram Frequency</li>
</ol>




In [4]:
#Extract features (mfcc, chroma, mel) from a given sound file(with path)
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

In [5]:
#Emotions in the RAVDESS dataset, it will not take the neutral into the modeling process
emotions={
  '01':'neutral',
  '02':'neutral',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fear',
  '07':'disgust',
  '08':'surprise'
}
#Emotions to observe(all emotions except neutral)
observed_emotions=['neutral', 'happy', 'sad','angry','fear', 'disgust','surprise']

## Retrieve RAVDESS dataset from File System

In [6]:
#audio file path in file system
RAV = "../datasets/RAVDESS/audio_speech_actors_01-24/"
# test to run one example 
dir_list = os.listdir(RAV+"Actor_01/")
dir_list[0:5]

['03-01-01-01-01-01-01.wav',
 '03-01-01-01-01-02-01.wav',
 '03-01-01-01-02-01-01.wav',
 '03-01-01-01-02-02-01.wav',
 '03-01-02-01-01-01-01.wav']

In [7]:
ref = pd.read_csv("../datasets/data_path.csv")
ref.head()

,labels,source,path
0,male_neutral,RAVDESS,../datasets/RAVDESS/audio_speech_actors_01-24/...
1,male_neutral,RAVDESS,../datasets/RAVDESS/audio_speech_actors_01-24/...
2,male_neutral,RAVDESS,../datasets/RAVDESS/audio_speech_actors_01-24/...
3,male_neutral,RAVDESS,../datasets/RAVDESS/audio_speech_actors_01-24/...
4,male_neutral,RAVDESS,../datasets/RAVDESS/audio_speech_actors_01-24/...


In [8]:
ref['path']

0       ../datasets/RAVDESS/audio_speech_actors_01-24/...
1       ../datasets/RAVDESS/audio_speech_actors_01-24/...
2       ../datasets/RAVDESS/audio_speech_actors_01-24/...
3       ../datasets/RAVDESS/audio_speech_actors_01-24/...
4       ../datasets/RAVDESS/audio_speech_actors_01-24/...
                              ...                        
8685             ../datasets/AudioWAV/1091_WSI_DIS_XX.wav
8686             ../datasets/AudioWAV/1091_WSI_FEA_XX.wav
8687             ../datasets/AudioWAV/1091_WSI_HAP_XX.wav
8688             ../datasets/AudioWAV/1091_WSI_NEU_XX.wav
8689             ../datasets/AudioWAV/1091_WSI_SAD_XX.wav
Name: path, Length: 8690, dtype: object

In [9]:
print(ref.labels.value_counts())

male_sad          767
male_disgust      767
male_angry        767
male_fear         767
male_happy        767
male_neutral      719
female_happy      696
female_disgust    696
female_fear       696
female_angry      696
female_sad        696
female_neutral    656
Name: labels, dtype: int64


In [10]:
#Load the data and extract features for each sound file
def load_data(test_size=0.25):
    x,y=[],[]
    for file in glob.glob(path):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, train_size= 0.75,random_state=9)

In [11]:
df = pd.DataFrame(columns=['feature'])

# loop feature extraction over the entire dataset
counter=0
for index,path in enumerate(ref.path):
    X, sample_rate = librosa.load(path
                                  , res_type='kaiser_fast'
                                  ,duration=2.5
                                  ,sr=44100
                                  ,offset=0.5
                                 )
    sample_rate = np.array(sample_rate)
    
    # mean as the feature. Could do min and max etc as well. 
    mfccs = np.mean(librosa.feature.mfcc(y=X, 
                                        sr=sample_rate, 
                                        n_mfcc=40),
                    axis=0)
    df.loc[counter] = [mfccs]
    counter=counter+1   

# Check a few records to make sure its processed successfully
print(len(df))
df.head()

8690


,feature
0,"[-21.385315, -21.385315, -21.385315, -21.38531..."
1,"[-19.669842, -19.838392, -20.179165, -18.40570..."
2,"[-21.391132, -21.391132, -21.391132, -21.39113..."
3,"[-21.469006, -21.469006, -21.469006, -21.46900..."
4,"[-22.837025, -22.837025, -22.837025, -22.83702..."


In [12]:
df = pd.concat([ref,pd.DataFrame(df['feature'].values.tolist())],axis=1)
df[:5]

,labels,source,path,0,1,2,3,4,5,6,...,206,207,208,209,210,211,212,213,214,215
0,male_neutral,RAVDESS,../datasets/RAVDESS/audio_speech_actors_01-24/...,-21.385315,-21.385315,-21.385315,-21.385315,-21.385315,-21.385315,-21.385315,...,-17.314789,-18.879824,-20.803364,-21.046108,-20.095325,-20.280087,-20.763123,-19.553648,-20.154835,-19.897285
1,male_neutral,RAVDESS,../datasets/RAVDESS/audio_speech_actors_01-24/...,-19.669842,-19.838392,-20.179165,-18.405704,-18.904381,-19.933283,-20.974010,...,-21.251734,-21.251734,-21.251734,-21.251734,-21.251734,-21.251734,-21.251734,-21.251734,-21.251734,-21.251734
2,male_neutral,RAVDESS,../datasets/RAVDESS/audio_speech_actors_01-24/...,-21.391132,-21.391132,-21.391132,-21.391132,-21.381531,-21.391132,-21.391132,...,-21.391132,-21.391132,-21.391132,-21.391132,-21.108055,-21.259764,-21.391132,-21.391132,-21.391132,-21.391132
3,male_neutral,RAVDESS,../datasets/RAVDESS/audio_speech_actors_01-24/...,-21.469006,-21.469006,-21.469006,-21.469006,-21.469006,-21.469006,-21.469006,...,-17.526384,-17.198935,-17.701351,-18.673058,-19.091045,-19.486828,-20.428762,-20.331993,-19.380220,-18.689539
4,male_neutral,RAVDESS,../datasets/RAVDESS/audio_speech_actors_01-24/...,-22.837025,-22.837025,-22.837025,-22.837025,-22.837025,-22.837025,-22.837025,...,-18.009171,-17.820364,-18.525299,-18.332527,-17.388416,-17.585821,-19.182173,-19.070602,-18.440105,-18.917660


In [13]:
df.head()

,labels,source,path,0,1,2,3,4,5,6,...,206,207,208,209,210,211,212,213,214,215
0,male_neutral,RAVDESS,../datasets/RAVDESS/audio_speech_actors_01-24/...,-21.385315,-21.385315,-21.385315,-21.385315,-21.385315,-21.385315,-21.385315,...,-17.314789,-18.879824,-20.803364,-21.046108,-20.095325,-20.280087,-20.763123,-19.553648,-20.154835,-19.897285
1,male_neutral,RAVDESS,../datasets/RAVDESS/audio_speech_actors_01-24/...,-19.669842,-19.838392,-20.179165,-18.405704,-18.904381,-19.933283,-20.974010,...,-21.251734,-21.251734,-21.251734,-21.251734,-21.251734,-21.251734,-21.251734,-21.251734,-21.251734,-21.251734
2,male_neutral,RAVDESS,../datasets/RAVDESS/audio_speech_actors_01-24/...,-21.391132,-21.391132,-21.391132,-21.391132,-21.381531,-21.391132,-21.391132,...,-21.391132,-21.391132,-21.391132,-21.391132,-21.108055,-21.259764,-21.391132,-21.391132,-21.391132,-21.391132
3,male_neutral,RAVDESS,../datasets/RAVDESS/audio_speech_actors_01-24/...,-21.469006,-21.469006,-21.469006,-21.469006,-21.469006,-21.469006,-21.469006,...,-17.526384,-17.198935,-17.701351,-18.673058,-19.091045,-19.486828,-20.428762,-20.331993,-19.380220,-18.689539
4,male_neutral,RAVDESS,../datasets/RAVDESS/audio_speech_actors_01-24/...,-22.837025,-22.837025,-22.837025,-22.837025,-22.837025,-22.837025,-22.837025,...,-18.009171,-17.820364,-18.525299,-18.332527,-17.388416,-17.585821,-19.182173,-19.070602,-18.440105,-18.917660


In [14]:
df=df.fillna(0)
print(df.shape)
df[:5]

(8690, 219)


,labels,source,path,0,1,2,3,4,5,6,...,206,207,208,209,210,211,212,213,214,215
0,male_neutral,RAVDESS,../datasets/RAVDESS/audio_speech_actors_01-24/...,-21.385315,-21.385315,-21.385315,-21.385315,-21.385315,-21.385315,-21.385315,...,-17.314789,-18.879824,-20.803364,-21.046108,-20.095325,-20.280087,-20.763123,-19.553648,-20.154835,-19.897285
1,male_neutral,RAVDESS,../datasets/RAVDESS/audio_speech_actors_01-24/...,-19.669842,-19.838392,-20.179165,-18.405704,-18.904381,-19.933283,-20.974010,...,-21.251734,-21.251734,-21.251734,-21.251734,-21.251734,-21.251734,-21.251734,-21.251734,-21.251734,-21.251734
2,male_neutral,RAVDESS,../datasets/RAVDESS/audio_speech_actors_01-24/...,-21.391132,-21.391132,-21.391132,-21.391132,-21.381531,-21.391132,-21.391132,...,-21.391132,-21.391132,-21.391132,-21.391132,-21.108055,-21.259764,-21.391132,-21.391132,-21.391132,-21.391132
3,male_neutral,RAVDESS,../datasets/RAVDESS/audio_speech_actors_01-24/...,-21.469006,-21.469006,-21.469006,-21.469006,-21.469006,-21.469006,-21.469006,...,-17.526384,-17.198935,-17.701351,-18.673058,-19.091045,-19.486828,-20.428762,-20.331993,-19.380220,-18.689539
4,male_neutral,RAVDESS,../datasets/RAVDESS/audio_speech_actors_01-24/...,-22.837025,-22.837025,-22.837025,-22.837025,-22.837025,-22.837025,-22.837025,...,-18.009171,-17.820364,-18.525299,-18.332527,-17.388416,-17.585821,-19.182173,-19.070602,-18.440105,-18.917660


In [15]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(['path','labels','source'],axis=1)
                                                    , df.labels
                                                    , test_size=0.25
                                                    , shuffle=True
                                                    , random_state=42
                                                   )

# Lets see how the data present itself before normalisation 
X_train[150:160]

,0,1,2,3,4,5,6,7,8,9,...,206,207,208,209,210,211,212,213,214,215
7065,-5.652812,-6.776311,-6.489865,-6.216996,-7.645125,-7.813805,-7.091050,-7.716411,-8.168571,-6.605966,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7462,-5.298156,-5.729014,-6.299941,-6.583365,-5.820536,-6.362069,-8.291990,-7.865595,-6.619760,-6.231025,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5928,-2.112594,-3.124799,-5.210888,-4.751945,-5.015863,-4.501850,-4.240977,-2.609289,-2.525379,-3.288661,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2184,-7.640528,-6.696494,-5.909721,-5.552992,-6.648856,-6.502574,-7.554166,-6.673454,-6.570912,-6.359321,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
110,-20.732349,-19.731350,-19.716396,-18.947292,-18.354311,-18.791056,-19.872158,-20.679012,-20.699242,-19.989124,...,-9.449995,-10.183277,-11.181304,-13.551791,-13.095141,-12.855795,-14.772998,-13.533300,-12.520718,-11.703916
6786,-6.944965,-6.288961,-7.191911,-7.211984,-7.731867,-7.288754,-7.856411,-9.319613,-8.113953,-7.637610,...,-5.991698,-5.924673,-5.700793,-6.681861,-6.341326,-6.599488,-6.353091,-6.232457,-6.694018,-6.857346
7111,-7.537225,-7.805920,-6.723403,-5.574308,-5.256060,-5.945969,-5.592881,-5.795800,-5.778611,-5.691808,...,-5.779315,-6.412852,-6.471704,-5.907022,-6.155215,-7.285172,-7.429246,-7.557392,-7.303209,-7.190961
1079,-17.302807,-17.139887,-17.887287,-18.465267,-18.293123,-17.757235,-17.011190,-16.545895,-17.135075,-17.434570,...,-10.292582,-10.875978,-9.645720,-9.881312,-9.332804,-9.620864,-10.634344,-10.021311,-9.222058,-7.555440
2226,-1.325179,-2.417265,-4.768901,-6.344377,-7.128631,-6.700863,-6.726776,-5.894978,-5.962801,-6.616137,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3297,-8.123964,-6.895119,-7.116265,-7.060562,-7.478992,-6.078900,-5.163329,-4.885925,-5.400164,-5.613014,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [16]:
mean = np.mean(X_train, axis=0)
std = np.std(X_train, axis=0)

X_train = (X_train - mean)/std
X_test = (X_test - mean)/std

# Check the dataset now 
X_train[150:160]

,0,1,2,3,4,5,6,7,8,9,...,206,207,208,209,210,211,212,213,214,215
7065,0.422619,0.185178,0.325245,0.387283,0.038118,-0.005075,0.165682,0.013389,-0.101318,0.273819,...,0.615444,0.594974,0.594440,0.595095,0.573142,0.573078,0.574060,0.558266,0.555101,0.551244
7462,0.500737,0.427829,0.371177,0.298782,0.478435,0.345211,-0.123594,-0.022474,0.270936,0.363819,...,0.615444,0.594974,0.594440,0.595095,0.573142,0.573078,0.574060,0.558266,0.555101,0.551244
5928,1.202402,1.031205,0.634558,0.741182,0.672622,0.794059,0.852193,1.241112,1.255015,1.070100,...,0.615444,0.594974,0.594440,0.595095,0.573142,0.573078,0.574060,0.558266,0.555101,0.551244
2184,-0.015203,0.203671,0.465549,0.547680,0.278541,0.311309,0.054129,0.264110,0.282677,0.333023,...,0.615444,0.594974,0.594440,0.595095,0.573142,0.573078,0.574060,0.558266,0.555101,0.551244
110,-2.898860,-2.816404,-2.873521,-2.687857,-2.546267,-2.653750,-2.912965,-3.102746,-3.113049,-2.938655,...,-1.088133,-1.233389,-1.406725,-1.818263,-1.767470,-1.718987,-2.041924,-1.831634,-1.686466,-1.550168
6786,0.138004,0.298094,0.155459,0.146933,0.017185,0.121613,-0.018674,-0.372011,-0.088191,0.026185,...,-0.464696,-0.468775,-0.425855,-0.594838,-0.560300,-0.603547,-0.550937,-0.542348,-0.643320,-0.679977
7111,0.007551,-0.053374,0.268765,0.542531,0.614657,0.445611,0.526553,0.475093,0.473105,0.493252,...,-0.426409,-0.556426,-0.563828,-0.456851,-0.527035,-0.725798,-0.741502,-0.776324,-0.752383,-0.739877
1079,-2.143455,-2.215982,-2.431161,-2.571418,-2.531501,-2.404302,-2.223830,-2.109168,-2.256406,-2.325463,...,-1.240028,-1.357760,-1.131895,-1.164609,-1.094994,-1.142230,-1.309057,-1.211437,-1.095912,-0.805318
2226,1.375841,1.195136,0.741451,0.356513,0.162760,0.263464,0.253426,0.451251,0.428836,0.271378,...,0.615444,0.594974,0.594440,0.595095,0.573142,0.573078,0.574060,0.558266,0.555101,0.551244
3297,-0.121687,0.157652,0.173753,0.183510,0.078209,0.413536,0.630021,0.693822,0.564065,0.512166,...,0.615444,0.594974,0.594440,0.595095,0.573142,0.573078,0.574060,0.558266,0.555101,0.551244


In [17]:
max_data = np.max(X_train)
min_data = np.min(X_train)
X_train = (X_train-min_data)/(max_data-min_data+1e-6)
X_train =  X_train-0.5

max_data = np.max(X_test)
min_data = np.min(X_test)
X_test = (X_test-min_data)/(max_data-min_data+1e-6)
X_test =  X_test-0.5

X_train[150:160]

,0,1,2,3,4,5,6,7,8,9,...,206,207,208,209,210,211,212,213,214,215
7065,0.215567,0.214376,0.253354,0.277585,0.216141,0.248841,0.250167,0.239985,0.231066,0.281149,...,0.500000,0.500000,0.500000,0.500000,0.480386,0.443441,0.447967,0.493151,0.500000,0.459790
7462,0.226781,0.249161,0.259919,0.264675,0.279451,0.301948,0.207657,0.234618,0.287354,0.294660,...,0.500000,0.500000,0.500000,0.500000,0.480386,0.443441,0.447967,0.493151,0.500000,0.459790
5928,0.327503,0.335659,0.297564,0.329209,0.307372,0.369998,0.351049,0.423728,0.436157,0.400684,...,0.500000,0.500000,0.500000,0.500000,0.480386,0.443441,0.447967,0.493151,0.500000,0.459790
2184,0.152719,0.217027,0.273408,0.300982,0.250710,0.296808,0.233774,0.277509,0.289130,0.290037,...,0.500000,0.500000,0.500000,0.500000,0.480386,0.443441,0.447967,0.493151,0.500000,0.459790
110,-0.261221,-0.215919,-0.203839,-0.170993,-0.155448,-0.152723,-0.202240,-0.226380,-0.224337,-0.201095,...,0.116402,0.084216,0.055217,-0.032775,-0.049838,-0.033921,-0.083893,-0.050809,-0.004711,0.017840
6786,0.174712,0.230563,0.229087,0.242524,0.213132,0.268048,0.223075,0.182306,0.233051,0.243975,...,0.256783,0.258095,0.273227,0.237309,0.223625,0.198388,0.219242,0.242642,0.230164,0.200850
7111,0.155986,0.180178,0.245282,0.300231,0.299038,0.317170,0.303197,0.309085,0.317924,0.314090,...,0.265404,0.238163,0.242561,0.267771,0.231161,0.172928,0.180498,0.189387,0.205607,0.188253
1079,-0.152785,-0.129845,-0.140614,-0.154008,-0.153325,-0.114905,-0.100972,-0.077680,-0.094804,-0.109045,...,0.082200,0.055933,0.116302,0.111526,0.102500,0.086199,0.065108,0.090352,0.128258,0.174490
2226,0.352400,0.359159,0.312842,0.273096,0.234063,0.289554,0.263061,0.305516,0.311230,0.280783,...,0.500000,0.500000,0.500000,0.500000,0.480386,0.443441,0.447967,0.493151,0.500000,0.459790
3297,0.137434,0.210430,0.231702,0.247860,0.221906,0.312307,0.318401,0.341820,0.331678,0.316929,...,0.500000,0.500000,0.500000,0.500000,0.480386,0.443441,0.447967,0.493151,0.500000,0.459790


In [18]:
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

# one hot encode the target 
lb = LabelEncoder()
y_train = np_utils.to_categorical(lb.fit_transform(y_train))
y_test = np_utils.to_categorical(lb.fit_transform(y_test))

print(X_train.shape)
print(lb.classes_)
#print(y_train[0:10])
#print(y_test[0:10])

# Pickel the lb object for future use 
filename = 'labels'
outfile = open(filename,'wb')
pickle.dump(lb,outfile)
outfile.close()

(6517, 216)
['female_angry' 'female_disgust' 'female_fear' 'female_happy'
 'female_neutral' 'female_sad' 'male_angry' 'male_disgust' 'male_fear'
 'male_happy' 'male_neutral' 'male_sad']


In [19]:
#Get the shape of the training and testing datasets
print((X_train.shape[0], X_test.shape[0]))

(6517, 2173)


In [20]:
#Get the number of features extracted
print(f'Features extracted: {X_train.shape[1]}')

Features extracted: 216


## MLP Classifier and Fit/Train the Model

In [21]:
#Initialize the Multi Layer Perceptron Classifier
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [22]:
#Fit/Train the model
model.fit(X_train,y_train)

C:\Users\User\anaconda3\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

## Predict the accuracy and result

In [23]:
#Predict for the test set
y_pred=model.predict(X_test)

In [26]:
#Calculate the accuracy of the model

accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)
# matrix = confusion_matrix(y_test,y_pred)
#Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

#Print the report
print("Statistics:")
print(classification_report(y_test,y_pred))

# #Print the Confusion Matrix
# print("Confusion Matrix:")
# df_cm = pd.DataFrame(matrix)
# sn.heatmap(df_cm, annot=True, fmt='')
# plt.show()

Accuracy: 5.02%
Statistics:
              precision    recall  f1-score   support

           0       0.19      0.12      0.15       160
           1       0.09      0.02      0.03       182
           2       0.00      0.00      0.00       160
           3       0.14      0.01      0.02       176
           4       0.10      0.11      0.11       159
           5       0.18      0.24      0.21       170
           6       0.29      0.18      0.23       192
           7       0.16      0.02      0.03       183
           8       0.14      0.07      0.09       184
           9       0.19      0.09      0.12       205
          10       0.33      0.02      0.03       197
          11       0.15      0.08      0.10       205

   micro avg       0.17      0.08      0.11      2173
   macro avg       0.16      0.08      0.09      2173
weighted avg       0.17      0.08      0.09      2173
 samples avg       0.06      0.08      0.07      2173



C:\Users\User\anaconda3\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
